In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import re
import pkg_resources

globalPrefixWordsSet = set()
globalRootWordsSet = set()
globalSuffixWordsSet = set()

data_path = '/content/drive/MyDrive/CollabData/Lemmatizer/FinalData/data.csv'
with open(data_path, encoding='utf-8') as f:
  reader=csv.reader(f)
  for row in reader:
    globalPrefixWordsSet.add(row[1])
    globalRootWordsSet.add(row[2])
    globalSuffixWordsSet.add(row[3])

  f.close()


directRoots = dict()

direct_path = '/content/drive/MyDrive/CollabData/Lemmatizer/FinalData/direct.csv'
with open(direct_path, encoding='utf-8') as f:
  reader=csv.reader(f)
  for row in reader:
    directRoots[row[0]] = row[1]

  f.close()


totalWords = set()

total_path = '/content/drive/MyDrive/CollabData/Lemmatizer/FinalData/totalWords.txt'
with open(total_path, encoding='utf-8') as f:
  data = f.readlines()
  for i in range(len(data)):
    totalWords.add(data[i].strip())
  f.close()


globalPrefixWordsSet.discard("")
globalRootWordsSet.discard("")
globalSuffixWordsSet.discard("")

In [ ]:
import re 
alreadyLemmatizedWords = dict()
# Here comes real algorithm
class WordLemmatizer:
  
  def __init__(self, word):
    self.word = word
    self.inflectedWord = list(word)
    self.possibleRootWords = set()
  
  def lemmatize(self):
    if (len(self.word)<3):
      '''
        done this because बुरा is giving बु which is not a root word, so length check is made
      '''
      return self.word

    return self.step1()
  
  def step1(self):

    for r in range(1,len(self.word)):
      tempWord = self.word[:r+1]
      if tempWord in globalRootWordsSet and len(tempWord)>1:
        self.possibleRootWords.add(tempWord)

    if(len(self.possibleRootWords)):
      return self.finalReturn()
    
    return self.step2()
  
  def step2(self):  

    for l in range(2,len(self.word)):
      if self.word[l:] in globalSuffixWordsSet and len(self.word[l:])>1 and (self.word[:l] in totalWords or self.word[:l] in globalRootWordsSet):
        self.possibleRootWords.add(WordLemmatizer(self.word[:l]).lemmatize())    
      
    if(len(self.possibleRootWords)):
      return self.finalReturn()

    return self.word

  def step3(self):

    for r in range(1,len(self.inflectedWord)-1):
      if self.word[:r] in globalPrefixWordsSet and (self.word[r:] in totalWords or self.word[r:] in globalRootWordsSet):
        self.possibleRootWords.add(WordLemmatizer(self.word[r:]).lemmatize())
    if(len(self.possibleRootWords)):
      return self.finalReturn()

    return self.word

  def finalReturn(self):
    # currently here returning longest one
    possibleRootWords = list(self.possibleRootWords)
    possibleRootWords.sort(key = lambda x: -len(x))

    if possibleRootWords and len(possibleRootWords[0])>2:
      return possibleRootWords[0]

    return self.word

def sanitizeLine(line):
  return re.sub(r'\s+', ' ', re.sub(r'[{}[\]:;?@!#$%^&*()-_\'\"<>\u0964\u0965,]', ' ', line.encode("utf-8").decode("utf-8"))).split()

def processWord(word):

  if word in alreadyLemmatizedWords:
    return alreadyLemmatizedWords[word]

  lemma= WordLemmatizer(word).lemmatize()
  alreadyLemmatizedWords[word]=lemma
  return lemma

def lemmatize(line):
  # this is the function that will be called by the user to lemmatize hindi text, it can accept both word and a line
  words = sanitizeLine(line)

  return " ".join([processWord(word) for word in words if word])

In [ ]:
# line = "वक्त की मार से हमेशा डर कर रहो क्योंकि बुरा वक्त कभी पूछ कर नहीं आता।"
# line = "जिन्दगी से जुड़ी सच्ची बातें"
line = "जो आपके सीखने की क्षमता है वह एक उपहार है, जो आपके सीखने की योग्यता है वह आपका कौशल है और आपके सीखने की इच्छा एक विकल्प है।"
line = "कार्यालय में विभिन्न विभागों की टीमें सहयोग करके समस्याओं का समाधान कर रही हैं। उनके योगदान से कंपनी के उद्देश्यों को पूरा करने में मदद मिलती है। आपूर्ति श्रृंखला को निरंतर सुधारते हुए, गुणवत्ता नियंत्रण प्रक्रिया को मजबूत किया जाता है। यह सुनिश्चित करने के लिए कि उत्पादों का मानक अनुरूप है और ग्राहकों को संतुष्ट किया जा सके। विपणन और प्रचार के माध्यम से, व्यापार का प्रशंसा किया जाता है और नए ग्राहकों को प्रवर्धित करने का प्रयास किया जाता है।"
print(lemmatize(line))


कार्यालय में विभिन्न विभाग की टीम सहयोग कर समस्या का समाधान कर रह है उनके योग से कंपनी के उद्देश्य को पूरा कर में मदद मिल है आपूर्ति श्रृंखला को निरंतर सुधार हो गुण नियंत्रण प्रक्रिया को मजबूत कर जा है यह सुन कर के ले कि उत्पाद का मान अनुरूप है और ग्राहक को संतुष्ट कर जा सक विपणन और प्र के माध्यम से व्यापार का प्रशंसा कर जा है और नए ग्राहक को प्रवर्ध कर का प्रयास कर जा है


In [ ]:
!pip install hindilemmatizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from hindilemmatizer import lemmatizer

print(lemmatizer.lemmatize("जो आपके सीखने की क्षमता है वह एक उपहार है, जो आपके सीखने की योग्यता है वह आपका कौशल है और आपके सीखने की इच्छा एक विकल्प है।"))

जो आपके सीख की क्षमता है वह एक उपह है जो आपके सीख की योग है वह आपका कौशल है और आपके सीख की इच्छा एक विकल्प है
